## 1. 实例
- 对于 Handler 的用法，往往是在一个线程中运行 Looper，其他线程通过 Handler 来发送消息到 Looper 所在线程，这里涉及线程间的通信

### 1.1 利用 HandlerThread 创建
- 在 HandlerThread 线程中运行 Loop() 方法，在其他线程中通过 Handler 发送消息到 HandlerThread 线程。通过 wait/notifyAll 的方式，有效地解决了多线程的同步问题

```
// Step 1: 创建并启动 HandlerThread 线程，内部包含 Looper
HandlerThread handlerThread = new HandlerThread("demo.com");
handlerThread.start();

// Step 2: 创建 Handler
Handler handler = new Handler(handlerThread.getLooper());

// Step 3: 发送消息
handler.post(new Runnable() {
    @Override
    public void run() {
        System.out.println("thread id="+Thread.currentThread().getId());
    }
});
```

### 1.2 直接创建线程
```
class LooperThread extends Thread {
    public Handler mHandler;

    public void run() {
        Looper.prepare();
        
        // Step 1: 创建 Handler
        mHandler = new Handler() {
            public void handleMessage(Message msg) {
                //处理即将发送过来的消息
                System.out.println("thread id="+Thread.currentThread().getId());
            }
        };

        Looper.loop();
    }
}

// Step 2: 创建并启动 LooperThread 线程，内部包含 Looper
LooperThread looperThread = new LooperThread("demo.com");
looperThread.start();

// Step 3: 发送消息
LooperThread.mHandler.sendEmptyMessage(10);
```

---

## HandlerThread 源码实现
### 1.1 创建 HandlerThread 对象
- HandlerThread 继承于 Thread 类

```
public HandlerThread(String name) {
    super(name);
    mPriority = Process.THREAD_PRIORITY_DEFAULT; // 默认优先级
}

public HandlerThread(String name, int priority) {
    super(name);
    mPriority = priority;
}
```

### 1.2 获取 Looper 对象
```
public Looper getLooper() {
    // 当线程没有启动或者已经结束时，则返回 null
    if (!isAlive()) {
        return null;
    }

    // 当线程已经启动，则等待直到 looper 创建完成
    synchronized (this) {
        while (isAlive() && mLooper == null) {
            try {
                wait(); // 休眠等待
            } catch (InterruptedException e) {
            }
        }
    }
    return mLooper;
}
```

### 1.3 执行 HandlerThread 的 run()
```
public void run() {
    mTid = Process.myTid();  // 获取线程的 tid
    Looper.prepare();        // 创建 Looper 对象
    synchronized (this) {
        mLooper = Looper.myLooper(); // 获取 looper 对象
        notifyAll();         // 唤醒等待线程
    }
    Process.setThreadPriority(mPriority);
    onLooperPrepared();  // 该方法可通过覆写，实现自己的逻辑
    Looper.loop();       // 进入循环模式
    mTid = -1;
}
```

### 1.4 Looper 退出
- quit() 与 quitSafely() 的区别，仅仅在于是否移除当前正在处理的消息。移除当前正在处理的消息可能会出现不安全的行为。

```
public boolean quit() {
    Looper looper = getLooper();
    if (looper != null) {
        looper.quit(); //普通退出
        return true;
    }
    return false;
}

public boolean quitSafely() {
    Looper looper = getLooper();
    if (looper != null) {
        looper.quitSafely(); //安全退出
        return true;
    }
    return false;
}
```

---